In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip install panadarallel

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randrange, uniform
pd.set_option('display.max_columns',None)
# from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import math
from sklearn.metrics import r2_score, mean_squared_error
# import pandarallel

In [ ]:
ride_share_df = pd.read_csv("/content/drive/MyDrive/ML Project/rideshare_kaggle.csv")

In [ ]:
ride_share_df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,product_id,name,price,distance,surge_multiplier,latitude,longitude,temperature,apparentTemperature,short_summary,long_summary,precipIntensity,precipProbability,humidity,windSpeed,windGust,windGustTime,visibility,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,icon,dewPoint,pressure,windBearing,cloudCover,uvIndex,visibility.1,ozone,sunriseTime,sunsetTime,moonPhase,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,lyft_line,Shared,5.0,0.44,1.0,42.2148,-71.033,42.34,37.12,Mostly Cloudy,Rain throughout the day.,0.0000,0.0,0.68,8.66,9.17,1545015600,10.000,43.68,1544968800,34.19,1545048000,37.95,1544968800,27.39,1545044400,partly-cloudy-night,32.70,1021.98,57,0.72,0,10.000,303.8,1544962084,1544994864,0.30,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,lyft_premier,Lux,11.0,0.44,1.0,42.2148,-71.033,43.58,37.35,Rain,"Rain until morning, starting again in the eve...",0.1299,1.0,0.94,11.98,11.98,1543291200,4.786,47.30,1543251600,42.10,1543298400,43.92,1543251600,36.20,1543291200,rain,41.83,1003.97,90,1.00,0,4.786,291.1,1543232969,1543266992,0.64,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,lyft,Lyft,7.0,0.44,1.0,42.2148,-71.033,38.33,32.93,Clear,Light rain in the morning.,0.0000,0.0,0.75,7.33,7.33,1543334400,10.000,47.55,1543320000,33.10,1543402800,44.12,1543320000,29.11,1543392000,clear-night,31.10,992.28,240,0.03,0,10.000,315.7,1543319437,1543353364,0.68,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,lyft_luxsuv,Lux Black XL,26.0,0.44,1.0,42.2148,-71.033,34.38,29.63,Clear,Partly cloudy throughout the day.,0.0000,0.0,0.73,5.28,5.28,1543514400,10.000,45.03,1543510800,28.90,1543579200,38.53,1543510800,26.20,1543575600,clear-night,26.64,1013.73,310,0.00,0,10.000,291.1,1543492370,1543526114,0.75,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,lyft_plus,Lyft XL,9.0,0.44,1.0,42.2148,-71.033,37.44,30.88,Partly Cloudy,Mostly cloudy throughout the day.,0.0000,0.0,0.70,9.14,9.14,1543446000,10.000,42.18,1543420800,36.71,1543478400,35.75,1543420800,30.29,1543460400,partly-cloudy-night,28.61,998.36,303,0.44,0,10.000,347.7,1543405904,1543439738,0.72,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


In [ ]:
ride_share_df['price'] = ride_share_df['price'].fillna(ride_share_df['price'].mean())

In [ ]:
# geolocator = Nominatim()
# ride_share_df['source_coord'] = ride_share_df['source'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
# ride_share_df['dest_coord'] = ride_share_df['destination'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [ ]:
ride_share_df.source.unique()

array(['Haymarket Square', 'Back Bay', 'North End', 'North Station',
       'Beacon Hill', 'Boston University', 'Fenway', 'South Station',
       'Theatre District', 'West End', 'Financial District',
       'Northeastern University'], dtype=object)

In [ ]:
ride_share_df.destination.unique()

array(['North Station', 'Northeastern University', 'West End',
       'Haymarket Square', 'South Station', 'Fenway', 'Theatre District',
       'Beacon Hill', 'Back Bay', 'North End', 'Financial District',
       'Boston University'], dtype=object)

In [ ]:
ride_share_df.timezone.unique()

array(['America/New_York'], dtype=object)

In [ ]:
loc_coord = {'Haymarket Square' : [42.3628, -71.0583],
             'North Station' : [42.3664, -71.0620],
             'Northeastern University' : [42.3398, -71.0892],
             'West End' : [42.4556, -75.0966],
             'South Station' : [42.3519, -71.0552],
             'Fenway' : [42.3519, -71.0552],
             'Theatre District' : [40.7590, -73.9845],
             'Beacon Hill' : [42.3588, -71.0707],
             'Back Bay' : [42.3503, -71.0810],
             'North End' : [42.3647, -71.0542],
             'Financial District' : [40.7077, -74.0083],
             'Boston University' : [42.3505, -71.1054]
}

In [ ]:
src_latitude = []
src_longitude = []
dest_latitude = []
dest_longitude = []

for i in range(len(ride_share_df)):
  src_latitude.append(loc_coord[ride_share_df['source'][i]][0])
  src_longitude.append(loc_coord[ride_share_df['source'][i]][1])
  dest_latitude.append(loc_coord[ride_share_df['destination'][i]][0])
  dest_longitude.append(loc_coord[ride_share_df['destination'][i]][1])

ride_share_df['source_latitude'] = src_latitude
ride_share_df['source_longitude'] = src_longitude
ride_share_df['destination_latitude'] = dest_latitude
ride_share_df['destination_longitude'] = dest_longitude

In [ ]:
rel_cols = ['datetime', 'hour', 'day', 'month', 'source', 'destination', 'distance', 'source_latitude', 'source_longitude', 'destination_latitude', 'destination_longitude']

In [ ]:
new_data = ride_share_df[rel_cols]
new_data.head(5)

,datetime,hour,day,month,source,destination,distance,source_latitude,source_longitude,destination_latitude,destination_longitude
0,2018-12-16 09:30:07,9,16,12,Haymarket Square,North Station,0.44,42.3628,-71.0583,42.3664,-71.062
1,2018-11-27 02:00:23,2,27,11,Haymarket Square,North Station,0.44,42.3628,-71.0583,42.3664,-71.062
2,2018-11-28 01:00:22,1,28,11,Haymarket Square,North Station,0.44,42.3628,-71.0583,42.3664,-71.062
3,2018-11-30 04:53:02,4,30,11,Haymarket Square,North Station,0.44,42.3628,-71.0583,42.3664,-71.062
4,2018-11-29 03:49:20,3,29,11,Haymarket Square,North Station,0.44,42.3628,-71.0583,42.3664,-71.062


In [ ]:
source_coordinates = new_data[["source_latitude", "source_longitude"]].values

In [ ]:
regions = MiniBatchKMeans(n_clusters = 15, batch_size = 10000).fit(np.array(source_coordinates))

In [ ]:
cluster_column = regions.predict(new_data[["source_latitude", "source_longitude"]])
new_data['src_cluster'] = cluster_column

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
DF = new_data.groupby(['datetime','src_cluster']).size().reset_index(name='count')

In [ ]:
DF['count'] = DF['count'].apply(lambda x :  (x / DF['count'].max()))
DF['dayofweek'] = pd.DatetimeIndex(DF['datetime']).dayofweek
DF['hour'] = pd.DatetimeIndex(DF['datetime']).hour
DF['day'] = pd.DatetimeIndex(DF['datetime']).day
DF['month'] = pd.DatetimeIndex(DF['datetime']).month

In [ ]:
DF.head(5)

,datetime,src_cluster,count,dayofweek,hour,day,month
0,2018-11-26 03:40:46,0,0.020408,0,3,26,11
1,2018-11-26 03:40:46,1,0.244898,0,3,26,11
2,2018-11-26 03:40:46,2,0.020408,0,3,26,11
3,2018-11-26 03:40:46,3,0.020408,0,3,26,11
4,2018-11-26 03:40:46,4,0.081633,0,3,26,11


In [ ]:
X = DF[['src_cluster', 'month', 'day', 'hour', 'dayofweek']]
Y = DF['count']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

In [ ]:
model1 = LinearRegression()
model1.fit(X_train, Y_train)
pred1 = model1.predict(X_test)

In [ ]:
print("R2 score: ", r2_score(Y_test, pred1))
print("RMSE: ", math.sqrt(mean_squared_error(Y_test, pred1)))

R2 score:  0.04133249413964779
RMSE:  0.06583058598498677


In [ ]:
model2 = RandomForestRegressor()
model2.fit(X_train, Y_train)
pred2 = model2.predict(X_test)

In [ ]:
print("R2 score: ", r2_score(Y_test, pred2))
print("RMSE: ", math.sqrt(mean_squared_error(Y_test, pred2)))

R2 score:  0.08585741948578807
RMSE:  0.06428367351555701


In [ ]:
model3 = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model3.fit(X_train, Y_train)
pred3 = model3.predict(X_test)

[08:13:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
print("R2 score: ", r2_score(Y_test, pred3))
print("RMSE: ", math.sqrt(mean_squared_error(Y_test, pred3)))

R2 score:  0.09416599178437801
RMSE:  0.0639908719776246
